In [2]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, MapType
from pyspark.sql.functions import split, from_json, col

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k


In [3]:
spark = SparkSession.builder \
            .appName("ProcessBronzeToSilver") \
            .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-30e81f2f-efbf-4edd-bd56-1e0dd9aa2c67;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz

In [5]:
topic = "finance.broker.transactions.customers"

In [8]:
customer_stream_df = spark.read \
                          .format("delta") \
                          .option("path", f'/home/jovyan/work/datalake/bronze/{topic}') \
                          .load() \
                          .selectExpr("CAST(value AS STRING)", "timestamp")

AnalysisException: `/home/jovyan/work/datalake/bronze/finance.broker.transactions.customers` is not a Delta table.

In [ ]:
customer_stream_df.show()

customer_stream_df.writeStream \
                  .format("console") \
                  .option("checkpointLocation", f'/home/jovyan/work/checkpoint/{topic}-processing') \
                  .start().awaitTermination()